In [ ]:
!pip install segmentation_models

---
### 라이브러리 임포트
---

In [ ]:
# 기본 모듈 임포트
import glob
import shutil
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# 이미지 관련 모듈 임포트
from PIL import Image
from tensorflow.keras.preprocessing.image import img_to_array

# 모델 관련 모듈 임포트
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, Flatten, Add, Dropout, ReLU, Conv2D, MaxPooling2D, UpSampling2D, concatenate

# 손실함수 임포트
from segmentation_models.losses import bce_dice_loss

---
###이미지 전처리
---
> 이미지 Crop 후 저장

In [ ]:
# 이미지 이름 리스트 저장
path = './data'
X_train_name = os.listdir(path=f'{path}/X_train')
y_train_name = os.listdir(path=f'{path}/y_train')
len(X_train_name),len(y_train_name)

In [ ]:
# 이미지 로드 함수
def image_load(size, path, name_list):
    img_list = []
    for idx, img in enumerate(sorted(name_list)):
        img = Image.open(f'{path}{img}').resize((size, size))
        img = img.convert('L')
        img_list.append(img)
    print(len(img_list))
    return img_list
    
def image_load_np(size, path, name_list):
    img_list = []
    for idx, img in enumerate(sorted(name_list)):
        img = Image.open(f'{path}{img}').resize((size, size))
        img = img.convert('L')
        img_list.append(img_to_array(img).squeeze())
    img_list = np.array(img_list)
    print(len(img_list))
    return img_list

In [ ]:
# 이미지 크롭 함수
def image_crop(infilename,save_path):     
    img=infilename
    (img_h,img_w)=img.size
    # crop 할 사이즈 : grid_w, grid_h
    grid_w=456
    grid_h=456
    
    # crop height
    range_w=(int)(img_w/grid_w)
    range_h=(int)(img_h/grid_h)
    print(range_w,range_h)
    
    i=0
    
    for w in range(range_w):
        for h in range(range_h):
            bbox = (h*grid_h, w*grid_w, (h+1)*(grid_h), (w+1)*(grid_w))
            print(h*grid_h, w*grid_w, (h+1)*(grid_h), (w+1)*(grid_w))
            
            # 가로 세로 시작, 가로 세로 끝
            crop_img = img.crop(bbox)
            
            # fname = "{}.png".format("{0:05d}".format(i))
            fname = "{}.png".format(i)
            savename = save_path + fname
            crop_img.save(savename)
            print('save file ' + savename + '....')
            i += 1

In [ ]:
# 학습할 암반 이미지를 Crop후 저장한다.
for idx, i in enumerate(train_list1):
    if __name__ == '__main__':
        image_crop(i,f'data/crop/X_train_crop/{idx}')

# 학습할 Label 이미지를 Crop후 저장한다.
for idx, i in enumerate(train_list2):
    if __name__ == '__main__':
        image_crop(i,f'data/crop/y_train_crop/{idx}')

---
### 이미지 FR 기법으로 증강
---

In [ ]:
# 이미지 이름 리스트 저장
size = 256
path = './data'
X_train_name_crop = os.listdir(path=f'{path}/crop/X_train_crop')
y_train_name_crop = os.listdir(path=f'{path}/crop/y_train_crop')
X_train_name = os.listdir(path=f'{path}/X_train')
y_train_name = os.listdir(path=f'{path}/y_train')
X_test_name = os.listdir(path=f'{path}/X_test')
y_test_name = os.listdir(path=f'{path}/y_test')
len(X_train_name),len(y_train_name),len(X_test_name),len(y_test_name),len(X_train_name_crop),len(y_train_name_crop)

In [ ]:
# 이미지 로드
X_train = image_load(size, './data/crop/X_train_crop/', X_train_name_crop)
y_train = image_load(size, './data/crop/y_train_crop/', y_train_name_crop)
train_list1 = image_load(size, './data/X_train/', X_train_name)
train_list2 = image_load(size, './data/y_train/', y_train_name)
X_test = image_load_np(size, './data/X_test/', X_test_name)
y_test = image_load_np(size, './data/y_test/', y_test_name)

In [ ]:
# Crop한 이미지와 원본 이미지를 합친다.
for idx, img in enumerate(train_list1):
    X_train.append(img)
for idx, img in enumerate(train_list2):
    y_train.append(img)

In [ ]:
# FR 증강 기법 함수
def FR_tec(img_list):
  FR_List = []
  for img in img_list:
    # 이미지를 90도씩 회전한다.
    for angle in range(-90, 181, 90) :
      rotate_img = img.rotate(angle)
      FR_List.append(img_to_array(rotate_img).squeeze())
      # 회전된 이미지를 반전시킨다.
      flip_img = rotate_img.transpose(Image.FLIP_LEFT_RIGHT)
      FR_List.append(img_to_array(flip_img).squeeze())
  FR_List = np.array(FR_List)
  print(FR_List.shape)
  return FR_List

In [ ]:
# 학습데이터를 FR 기법을 통해 증강시킨다.
X_train = FR_tec(X_train)
y_train = FR_tec(y_train)

---
### 이미지를 npy형식으로 저장
---

In [ ]:
# 저장을 위해 하나의 튜플로 생성한다.
xy = (X_train, X_test, y_train, y_test)

# npy형식으로 저장
np.save('./data/data256_FRC+_5000.npy', xy)

---
### 데이터 로드
---
> data 아래의 디렉터리인 test, train을 불러옵니다.

In [ ]:
# 증강시킨 데이터를 가져온다
X_train, X_test, y_train, y_test = np.load('./data/data256_FRC+_5000.npy', allow_pickle=True)

# 효율적인 학습을 위해 스케일링한다
X_train = X_train/255.
X_test = X_test/255.
y_train = y_train/255.
y_test = y_test/255.

---
### 모델 만들기
---

In [ ]:
# Unet 모델 생성
def UNet(pretrained_weights = None,input_size = (256,256,1)):
    inp = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inp, outputs=[conv10])
    return model

In [ ]:
# 모델 컴파일
model = UNet()
model.summary()
lr = 1e-4
model.compile(optimizer = Adam(learning_rate = lr), loss = [bce_dice_loss], metrics = ['accuracy'])

In [ ]:
# 모델 저장 및 자동중지
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 
callbacks = [ModelCheckpoint(filepath='./model/last_model_unet_5000_hybrid.h5', monitor='val_loss', save_best_only=True), EarlyStopping(monitor='val_loss', patience=50)]

In [ ]:
# 학습한다
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs = 10000, batch_size = 16, verbose = 1)

In [ ]:
# 학습 history 리스트를 추출한다.
loss_list = history.history['loss']
accuracy_list = history.history['accuracy']
val_loss_list = history.history['val_loss']
val_accuracy_list = history.history['val_accuracy']

In [ ]:
# 정확도 그래프
plt.figure(figsize=(20,10))
plt.plot(accuracy_list, label='accuracy')
plt.plot(val_accuracy_list, label = 'val_accuracy')
plt.legend()
plt.show()

In [ ]:
# 손실율 그래프
plt.figure(figsize=(20,10))
plt.plot(loss_list, label='loss')
plt.plot(val_loss_list, label = 'val_loss')
plt.legend()
plt.show()